<a href="https://colab.research.google.com/github/DSWoodin/Pythonworkspace/blob/main/ProDS/%EB%B6%84%EB%A5%98_%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
from statsmodels.api import Logit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [41]:
df = pd.read_csv('iris.csv')
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [42]:
df['Species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [43]:
df['is_setosa'] = (df['Species'] == 'setosa') + 0
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1


In [44]:
model = Logit(endog=df['is_setosa'], exog=df.iloc[:, :2]).fit()
model

Optimization terminated successfully.
         Current function value: 0.036374
         Iterations 11


In [45]:
model.params

Sepal.Length    -7.529945
Sepal.Width     13.130734
dtype: float64

In [46]:
model.pvalues

Sepal.Length    0.000828
Sepal.Width     0.000989
dtype: float64

In [47]:
pred = model.predict(df.iloc[:3, :2])
pred

0    0.999477
1    0.923824
2    0.998678
dtype: float64

In [48]:
(pred > 0.5) + 0

0    1
1    1
2    1
dtype: int64

In [49]:
model = LogisticRegression(random_state=123)
model.fit(X=df.iloc[:, :2], y=df['is_setosa'])
model

LogisticRegression(random_state=123)

In [50]:
model.coef_

array([[-3.38829757,  3.1645277 ]])

In [51]:
model.intercept_

array([8.32330389])

In [52]:
pred = model.predict_proba(df.iloc[:3, :2])
pred = pred[:, 1]
pred

array([0.89272024, 0.77104635, 0.92586179])

In [53]:
(pred > 0.5) + 0

array([1, 1, 1])

In [54]:
pred = model.predict_proba(df.iloc[:, :2])
pred = pred[:, 1]
pred[:10]

array([0.89272024, 0.77104635, 0.92586179, 0.92738323, 0.94126096,
       0.91436651, 0.97058885, 0.89484454, 0.93034007, 0.82210603])

In [55]:
from sklearn.metrics import roc_auc_score

In [56]:
roc_auc_score(y_true=df['is_setosa'], y_score=pred)

0.9999999999999999

In [57]:
accuracy_score(y_true=df['is_setosa'], y_pred=(pred > 0.5) + 0)
# precision_score, recall_score, f1_score -> 사용법 비슷

1.0

### 문제 1: 독립변수를 혈압, 혈당, BMI, 인슐린으로 하고 종속변수를 당뇨 여부로 할 경우 분류 정확도는 얼마인가?

In [58]:
df = pd.read_csv('diabetes.csv')
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [59]:
from sklearn.model_selection import train_test_split

In [60]:
df_train, df_test = train_test_split(df, train_size=0.8, random_state=123)
df_train.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
318,3,115,66,39,140,38.1,0.150,28,0
313,3,113,50,10,85,29.5,0.626,25,0


In [63]:
model = Logit(endog = df_train['Outcome'],
              exog = df_train.loc[:, ['BloodPressure','Glucose','BMI','Insulin']]).fit()

Optimization terminated successfully.
         Current function value: 0.626579
         Iterations 5


In [65]:
pred = model.predict(exog = df_test.loc[:, ['BloodPressure','Glucose','BMI','Insulin']])
pred[:4]

236    0.462956
395    0.507051
36     0.359735
210    0.314389
dtype: float64

In [66]:
pred_class = (pred > 0.5) + 0
pred_class[:4]

236    0
395    1
36     0
210    0
dtype: int64

In [67]:
accuracy_score( y_pred = pred_class, y_true = df_test['Outcome'])

0.7012987012987013

### 문제 2: 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 나이의 승산비는 얼마인가?

In [68]:
df = pd.read_csv('diabetes.csv')
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [69]:
model = Logit(endog = df['Outcome'],
              exog = df.loc[:, ['Glucose','BMI','Age']]).fit()
model.params

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


Glucose    0.009368
BMI       -0.035639
Age       -0.012898
dtype: float64

In [71]:
np.exp(model.params)

Glucose    1.009412
BMI        0.964989
Age        0.987184
dtype: float64

### 문제 3: 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 모델의 AUC는 얼마인가?

In [72]:
df = pd.read_csv('diabetes.csv')
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [73]:
model = Logit(endog=df['Outcome'],
              exog=df.loc[:, ['Glucose','BMI','Age']]).fit()

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


In [74]:
model.predict(df.loc[:, ['Glucose','BMI','Age']])[:5]

0    0.387961
1    0.365506
2    0.615678
3    0.392087
4    0.336654
dtype: float64

In [75]:
roc_auc_score(y_true=df['Outcome'],
              y_score=model.predict(df.loc[:, ['Glucose','BMI','Age']]))

0.5414253731343283